# Faster–RCNN

In [9]:
import os
import random
import imageio
image_dir = '/workspaces/breast-tumor-detection/Data/Original_Data/pos'
images = os.listdir(image_dir)

In [8]:
select_images = random.choices(images, k = 40)
random.shuffle(select_images)
training_images = select_images[:30]
validation_images = select_images[30:]

In [14]:
#store training and validation images
train_destination_dir = '/workspaces/breast-tumor-detection/Model/FasterRCNN/data/train'
val_destination_dir = '/workspaces/breast-tumor-detection/Model/FasterRCNN/data/val'

#iterate to store
for img_path in training_images:
    img_path_full = os.path.join(image_dir, img_path)
    assert os.path.exists(img_path_full)
    img = imageio.v2.imread(img_path_full)
    imageio.imwrite(os.path.join(train_destination_dir, img_path), img)

In [15]:
#iterate to store
for img_path in validation_images:
    img_path_full = os.path.join(image_dir, img_path)
    assert os.path.exists(img_path_full)
    img = imageio.v2.imread(img_path_full)
    imageio.imwrite(os.path.join(val_destination_dir, img_path), img)